### **Product Information Chatbot using Langchain 🦜 and Comet ☄ :**:

The Product Information Chatbot is a conversational interface designed to provide users with information about products. It informs on the details of various products, such as specifications, prices, and reviews.

#### **Langchain**

#### Preliminary Setup
Before starting make sure to register an account on comet. The account provides you with the API keys to access all the goodness comet has to serve.

#### Setup and Imports

In [ ]:
!pip install langchain openai comet_llm textstat tiktoken --quiet

In [ ]:
import comet_llm
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

#### Making a simple Chatbot from Prompt Engineering 🧰
The section will include the how-to of creating a simple chatbot using langchain with simple prompt engineering. The chatbot will be able to remember the previous conversation and answer the queries.

##### 1. Chat Models 🗨
Chat models are backed by large language models. LLMs are different such that they are completion models while chat models are specifically tuned for having conversation. In this, we will be using **GPT-3.5**. There are many chat models supported in langchain. The list can be found [here](https://python.langchain.com/docs/integrations/chat/).

In [ ]:
llm = ChatOpenAI(temperature = 0.4,
                 openai_api_key="...")

##### 2. Prompt 🧑

Prompt engineering tailors chatbots from generalization to specialization. Langchian provides prompt templates that simplifies the creation of prompts by combining default messages, user input, chat history, and, optionally, additional context retrieved during the conversation.

`ChatPromptTemplate` takes a list of `MessagePromptTemplate`. LangChain provides different types of MessagePromptTemplate.
The most commonly used are `AIMessagePromptTemplate`, `SystemMessagePromptTemplate` and `HumanMessagePromptTemplate`, which create an AI message, system message and human message respectively.


In [ ]:
instructions = """You are a friendly chatbot capable of answering questions related to products. User's can ask questions about its specifications,
            prices and reviews. Be polite and redirect conversation specifically to product information when necessary."""

human = "Chat history of the user: {chat_history}\nNew human question: {input}"

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(instructions),
        # The `variable_name` here is what must align with memory
        HumanMessagePromptTemplate.from_template(human), #User query will go here
    ],

    input_variables=['chat_history','input'],
)


##### 3. Memory 📝
Memory allows the conversation to be stored and adds it to the current query for reference of the previous conversation. There are many memories supported by Langchain. The most common ones are as follows:

* `ConversationBufferMemory`: Simplest type of Memory

* `ConversationBufferWindowMemory`: keeps a list of the interactions of the conversation over time.

* `ConversationSummaryMemory`: Summary is created for the stored conversation. This condenses the information and reduces the chances of token limit problems.

* `ConversationTokenBufferMemory`: Truncates the stored messages if the token limit is reached as given by the user.

Our chatbot will keep the track of last 4 interactions.

In [ ]:
memory = ConversationBufferWindowMemory(memory_key = "chat_history",k = 4) #this will keep track of prev conversation

#### 4. Chains:
Chains is a simple concept of connecting different pieces like language model, prompt, memory etc into a single chain. There are two ways to achieve it. One is the legacy way through `LLMChain` while the other is LangChain Expression Language (LCEL). Here we

In [ ]:
conversation = ConversationChain(llm = llm,prompt = prompt, memory = memory, verbose = True) # LLMChain
conversation

In [ ]:
conversation.run("Which phone is better Samsung or Apple?")

In [ ]:
conversation.run("Compare them with onePlus?")

#### 5. Documents
We have created the chatbot, Now we need to pass in document to the chatbot for lookup and answer the product related question including reviews, product specifications and prices. Let's import the dataset from data.world which inlcudes Amazon Products. Here's the [link](https://query.data.world/s/76kyenosebb7rtcruaarjgnfogum66?dws=00000) to the dataset.

* Importing Data

In [ ]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/76kyenosebb7rtcruaarjgnfogum66?dws=00000')

df.to_csv('product.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
df.head(5)

* Loading Data

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = CSVLoader(file_path='product.csv',csv_args={
                'delimiter': ','})

data = loader.load()

* Splitting Data

In [ ]:
Splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 150)
splits = Splitter.create_documents([datum.page_content for datum in data])

##### 6. Chat Retreival
In order to chat with the documents or some other source of knowledge (in our case its the product CSV), RAG (Retrieval Augmented Generation) is used. RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
             openai_api_key="...") # Api Key

vectorstore = FAISS.from_documents(splits, embeddings)

* Retrieve:

  LangChain establishes a Retriever interface that encapsulates an index capable of providing pertinent documents in response to a textual query. All retrievers uniformly implement the method `get_relevant_documents()` and its asynchronous counterpart, `aget_relevant_documents()`.

We also need to input the context to the prompt for the chain to work.

In [ ]:
instructions = """You are a friendly chatbot capable of answering questions related to products. User's can ask questions about its specifications,
            prices and reviews. Be polite and redirect conversation specifically to product information when necessary."""

human = """
The context is provided as: {context}
New human question: {question}
"""
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(instructions),
        # The `variable_name` here is what must align with memory
        HumanMessagePromptTemplate.from_template(human), #User query will go here
    ],
    input_variables=['context','question'],
)

* Chain:
  
  Langchain conversation Retrieval chain is very useful to cater memory, retriever and prompt altogether.


In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa = ConversationalRetrievalChain.from_llm(llm = llm,
                                           memory=memory,
                                           get_chat_history=lambda x : x,
                                           retriever=retriever,
                                           combine_docs_chain_kwargs={"prompt": prompt}
                                           )

In [ ]:
qa

In [ ]:
def predict(question):

  ai_msg = qa({"question":question})['answer']
  return ai_msg

In [ ]:
predict("Hi,my name is Bob?")

In [ ]:
predict("What can you do?")

In [ ]:
predict("What is the best product?")

In [ ]:
predict("What's the price kindle paperwhite?")

#### Comet
When creating applications with LLMs, the majority of the time is spent on prompt engineering rather than training the models. This brings a new term in town **LLMOps**. Comet has a rich set of features for LLMOps namely:

* LLM Projects: It is designed for analyzing prompts, responses and chaining.

* LLM Panels: Visualizations compatible with Experiment Management can be employed to observe prompts and chains, particularly beneficial for projects involving both fine-tuning and prompt engineering use-cases.

Initially, it is required to create an account on [**Comet**](https://www.comet.com/signup) and then get the access of the api key to access its features.

In [ ]:

#COMET_WORKSPACE = "COMET_WORKSPACE"
PROJECT_NAME = "Product-Bot-v2"
# initialize comet
comet_llm.init(project=PROJECT_NAME)

In [ ]:
comet_llm.is_ready()

#### How to monitor and track on comet
Now in order to track the outputs of the conversation bot and check whether the prompt is good for your use case, comet has a simple function `log_prompt`:

log_prompt takes in the prompt or user query, prompt_template takes in the template and output takes in the output from the model.
metadata can take a dict of various properties for example token usage.

In [ ]:
queries = ["Hi,my name is Bob?","What can you do?","What is the best product?","What's the price kindle paperwhite?"]
expected_response = ["Hello, how can you assist you.",
              "As a chatbot, my capabilities include answering questions about product specifications, prices, and reviews. \
               I can provide information about various products and assist you in finding the information you need. \
               If you have any specific questions or need assistance with a particular product, feel free to ask.",
              "Based on the information provided, it seems that there is no single clear winner among the tablets mentioned.\
               Each tablet has its own strengths and weaknesses. The Amazon HDX and Nexus are praised for their pricing, \
               while Apple and Google have more app choices. If you are heavily invested in the Apple ecosystem, \
               the iPad Mini might be a good choice. Ultimately, the best product will depend on your personal preferences and requirements.",
              "The base model of the Kindle Paperwhite is priced at $99."]

#for index, convo in enumerate(queries):
    # log the few-shot predictions

comet_llm.log_prompt(
  prompt=queries[0],
  prompt_template= instructions,
  output = predict(queries[0]),
  tags = ["gpt-3.5-turbo", "prompt_1"],
  metadata = {
    "expected_answer": expected_response[0]
    },
)
